# MLFLow Trackingへの機械学習プロセスの組み込み（２）

先ほどは疑似データセットapple_dataを用いた実験結果の格納について確認した。  
先ほどは実行処理を単体として確認したが、ここでは実行処理の単位をネストして保存することが可能であることを確認する。  

ここではハイパーパラメータ`max_depth`の値のリスト`max_depths`を用意し、その値を順次実行して、その実行した結果をネストした単位で保存を行うことで確認する。

## 学習から評価まで

このフェーズは前のNotebookと同様である。

ここではapple_dataが格納されているcsvを取り込み、学習と検証の分割を行う。  
その分割されたデータを用いて、学習を行う関数`objective()`を定義する。  
この`objective()`をMLFlow Trackingのプロセスに組み込んでいく。

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('apple_data.csv', parse_dates=[0])
data

,date,average_temperature,rainfall,weekend,holiday,price_per_kg,promo,demand,previous_days_demand,competitor_price_per_kg,marketing_intensity
0,1996-12-02 23:45:11.240034,30.584727,1.831006,0,0,1.578387,1,1001.647352,1026.324266,0.755725,0.323086
1,1996-12-03 23:45:11.240032,15.465069,0.761303,0,0,1.965125,0,843.972638,1026.324266,0.913934,0.030371
2,1996-12-04 23:45:11.240031,10.786525,1.427338,0,0,1.497623,0,890.319248,868.942267,2.879262,0.354226
3,1996-12-05 23:45:11.240030,23.648154,3.737435,0,0,1.952936,0,811.206168,889.965021,0.826015,0.953000
4,1996-12-06 23:45:11.240029,13.861391,5.598549,0,0,2.059993,0,822.279469,835.253168,1.130145,0.953000
...,...,...,...,...,...,...,...,...,...,...,...
9995,2024-04-14 23:45:11.226286,23.358868,7.061220,1,0,1.556829,1,2566.432998,2676.279445,0.560507,0.889971
9996,2024-04-15 23:45:11.226284,14.859048,0.868655,0,0,1.632918,1,2032.827646,2590.543027,2.460766,0.884467
9997,2024-04-16 23:45:11.226283,17.941035,13.739986,0,0,0.827723,1,2167.417581,2031.943179,1.321922,0.884467
9998,2024-04-17 23:45:11.226281,14.533862,1.610512,0,0,0.589172,1,2099.505096,2166.533113,2.604095,0.812706


In [3]:
# データを特徴とターゲットに分割し、無関係な日付フィールドとターゲットフィールドをドロップする。
X = data.drop(columns=["date", "demand", "competitor_price_per_kg", "marketing_intensity"])
y = data["demand"]

# データを訓練セットと検証セットに分割する
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y
    , test_size=0.2
    , random_state=42
)

In [4]:
params = {
    "n_estimators": 100,
    #"max_depth": 6,
    "min_samples_split": 10,
    "min_samples_leaf": 4,
    "bootstrap": True,
    "oob_score": False,
}

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def objective(X_train, X_val, y_train, y_val, params):
    # RandomForestRegressor を訓練→fit→予測
    model_rf = RandomForestRegressor(**params)
    
    model_rf.fit(X_train, y_train)
    y_pred = model_rf.predict(X_val)

    # エラーメトリクスの計算
    mae = mean_absolute_error(y_val, y_pred)
    mse = mean_squared_error(y_val, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_val, y_pred)
    
    # これから書くメトリクスをコレクションにまとめる
    metrics = {
          "mae": mae
        , "mse": mse
        , "rmse": rmse
        , "r2": r2
    }
    
    return model_rf, metrics

## MLFlowの実行

### 初期設定

In [6]:
import mlflow

Trackingサーバへの接続を行う。

In [7]:
mlflow.set_tracking_uri("http://localhost:5000")

エクスペリメントおよび実行処理名を定義する。  
特に実行処理名はここでは固定で付与するものとする。

In [8]:
# エクスペリメントの定義
apple_experiment = mlflow.set_experiment("Apple-Models")
apple_experiment

<Experiment: artifact_location='mlflow-artifacts:/988028568725486117', creation_time=1713941969999, experiment_id='988028568725486117', last_update_time=1713941969999, lifecycle_stage='active', name='Apple_Models', tags={}>

In [9]:
# 実行処理名の定義
run_name = "apples_rf_test_nested"

つぎにモデルを保存するアーティファクトパスを以下の通り定義する。

In [10]:
artifact_path = "rf_apples"

ここでは`max_depth`の値を変化させながら実行させ、出力がネストされるようにする。

In [12]:
# max_depthのリストをここで定義する
max_depths = [2, 4, 8, 16]

# MLflowの実行コンテキストを開始する
with mlflow.start_run(run_name=run_name) as parent_run:
    # Create a child run for each parameter setting
    for p in max_depths:
        with mlflow.start_run(run_name="maxdepth : " + str(p), nested=True) as child_run:

            # 学習とその結果の取得
            model_rf, metrics = objective(X_train, X_val, y_train, y_val, params)
        
            # モデルフィットに使用されたパラメータを記録する
            mlflow.log_params(params)
            # max_depthは個別に記録
            mlflow.log_param("max_depth", p)
        
            # 検証中に計算されたエラーメトリックをログに記録する
            mlflow.log_metrics(metrics)
        
            # 後で使用するために学習済みモデルのインスタンスをログに記録する
            mlflow.sklearn.log_model(
                  sk_model=model_rf
                , input_example=X_val
                , artifact_path=artifact_path
            )

/home/user01/.local/lib/python3.10/site-packages/mlflow/types/utils.py:393: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
/home/user01/.local/lib/python3.10/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesi

MLFlow UIより、今回の実験の結果を確認する。  
今回のケースは実行処理がネストされて出力されている様子について確認して欲しい。